# 🇨🇱 Análisis Exploratorio de la Fuerza de Trabajo en Chile

## Análisis Avanzado de Datos del INE

**Autor:** Bruno San Martin  
**Fecha:** Julio 2025  
**Fuente:** Instituto Nacional de Estadísticas (INE) - Chile  

---

### 📊 Objetivos del Análisis

1. **Exploración de Datos**: Entender la estructura y calidad de los datos de fuerza laboral
2. **Análisis Temporal**: Identificar tendencias y patrones a lo largo del tiempo
3. **Análisis Regional**: Comparar indicadores entre diferentes regiones de Chile
4. **Análisis por Género**: Examinar diferencias en participación laboral por sexo
5. **Insights Económicos**: Generar conclusiones actionables para stakeholders

### 🎯 Metodología

- **Clean Code**: Código limpio y bien documentado
- **SOLID Principles**: Arquitectura modular y escalable  
- **Data Science Best Practices**: Validación, logging y reproducibilidad
- **Professional Visualizations**: Gráficos de calidad ejecutiva

## 1. 🛠️ Configuración del Entorno y Librerías

Configuración inicial del entorno de trabajo, importación de librerías necesarias y configuración de rutas del proyecto siguiendo arquitectura modular.

In [ ]:
# Configuración del entorno
import sys
import warnings
from pathlib import Path

# Añadir el directorio src al path para importar nuestros módulos
project_root = Path.cwd().parent
sys.path.append(str(project_root / "src"))

# Suprimir warnings innecesarios
warnings.filterwarnings('ignore')

print(f"📁 Directorio del proyecto: {project_root}")
print(f"🐍 Python path configurado correctamente")

In [ ]:
# Librerías principales para Data Science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Librerías para análisis estadístico
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Librerías del proyecto (arquitectura modular)
try:
    from etl.data_processor import LabourForceProcessor
    from visualization.charts import LabourForceCharts, StatisticalCharts
    from utils.validators import LabourForceValidator
    from utils.logger_config import get_logger
    print("✅ Módulos del proyecto importados correctamente")
except ImportError as e:
    print(f"⚠️  Error importando módulos del proyecto: {e}")
    print("📋 Continuaremos con librerías estándar")

# Configuración de visualizaciones
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Configuración de plotly
import plotly.io as pio
pio.templates.default = "plotly_white"

print("📊 Configuración de visualización completada")
print("🎯 Entorno listo para análisis")

## 2. 📥 Extracción y Organización de Datos del INE

Carga y exploración inicial de los datos oficiales de la Fuerza de Trabajo proporcionados por el Instituto Nacional de Estadísticas (INE) de Chile.

In [ ]:
# Definir rutas de datos
raw_data_path = project_root / "data" / "raw"
processed_data_path = project_root / "data" / "processed"
output_data_path = project_root / "data" / "outputs"

# Crear directorios si no existen
for path in [raw_data_path, processed_data_path, output_data_path]:
    path.mkdir(parents=True, exist_ok=True)

print(f"📂 Directorio de datos crudos: {raw_data_path}")
print(f"📂 Directorio de datos procesados: {processed_data_path}")
print(f"📂 Directorio de salida: {output_data_path}")

# Listar archivos disponibles en data/raw
raw_files = list(raw_data_path.glob("*.csv"))
print(f"\n📋 Archivos CSV disponibles: {len(raw_files)}")
for file in raw_files:
    file_size = file.stat().st_size / (1024*1024)  # MB
    print(f"   📄 {file.name} ({file_size:.2f} MB)")

In [ ]:
# Cargar datos del INE
if raw_files:
    # Tomar el primer archivo CSV encontrado
    data_file = raw_files[0]
    print(f"📊 Cargando datos desde: {data_file.name}")
    
    # Cargar datos
    try:
        df_raw = pd.read_csv(data_file, encoding='utf-8')
        print(f"✅ Datos cargados exitosamente")
        print(f"📐 Dimensiones: {df_raw.shape}")
        
    except UnicodeDecodeError:
        # Intentar con encoding alternativo
        df_raw = pd.read_csv(data_file, encoding='latin-1')
        print(f"✅ Datos cargados con encoding latin-1")
        print(f"📐 Dimensiones: {df_raw.shape}")
        
else:
    print("❌ No se encontraron archivos CSV en data/raw")
    print("📋 Por favor, coloca el archivo de datos del INE en la carpeta data/raw")

In [ ]:
# Exploración inicial de los datos
if 'df_raw' in locals():
    print("🔍 EXPLORACIÓN INICIAL DE DATOS")
    print("=" * 50)
    
    # Información básica
    print(f"📊 Forma del dataset: {df_raw.shape}")
    print(f"📋 Columnas: {list(df_raw.columns)}")
    
    # Mostrar primeras filas
    print("\n📖 Primeras 5 filas:")
    display(df_raw.head())
    
    # Información de tipos de datos
    print("\n🔢 Información de tipos de datos:")
    display(df_raw.info())
    
    # Estadísticas descriptivas
    print("\n📈 Estadísticas descriptivas:")
    display(df_raw.describe(include='all'))
    
    # Valores únicos en columnas categóricas
    print("\n🏷️ Valores únicos en columnas principales:")
    categorical_cols = df_raw.select_dtypes(include=['object']).columns
    for col in categorical_cols[:5]:  # Mostrar solo las primeras 5
        unique_count = df_raw[col].nunique()
        print(f"   {col}: {unique_count} valores únicos")
        if unique_count <= 10:
            print(f"      Valores: {list(df_raw[col].unique())}")
        else:
            print(f"      Algunos valores: {list(df_raw[col].unique()[:5])}...")
else:
    print("⚠️ Datos no disponibles para exploración")

## 3. 🧹 Procesamiento y Limpieza de Datos (ETL)

Aplicación de técnicas de ETL (Extract, Transform, Load) para limpiar, estandarizar y transformar los datos crudos del INE en un formato óptimo para análisis.

In [ ]:
# Validación de datos usando nuestro módulo personalizado
if 'df_raw' in locals():
    print("🔍 VALIDACIÓN DE CALIDAD DE DATOS")
    print("=" * 50)
    
    # Crear validador
    try:
        validator = LabourForceValidator()
        
        # Ejecutar validaciones
        validation_results = validator.validate_labour_force_data(df_raw)
        
        # Mostrar reporte
        print("\n📋 REPORTE DE VALIDACIÓN:")
        report = validator.get_validation_report()
        print(report)
        
    except NameError:
        print("⚠️ Usando validación básica (módulo personalizado no disponible)")
        
        # Validación básica manual
        print(f"✅ Datos no vacíos: {not df_raw.empty}")
        print(f"✅ Sin filas completamente vacías: {df_raw.dropna(how='all').shape[0] == df_raw.shape[0]}")
        print(f"📊 Valores faltantes por columna:")
        missing_data = df_raw.isnull().sum()
        missing_data = missing_data[missing_data > 0]
        if len(missing_data) == 0:
            print("   🎉 No hay valores faltantes")
        else:
            for col, count in missing_data.items():
                percentage = (count / len(df_raw)) * 100
                print(f"   {col}: {count} ({percentage:.2f}%)")
    
    print("\n" + "="*50)

In [ ]:
# Procesamiento de datos usando arquitectura modular
if 'df_raw' in locals():
    print("🔄 PROCESAMIENTO DE DATOS")
    print("=" * 50)
    
    try:
        # Usar procesador personalizado
        processor = LabourForceProcessor()
        
        # Procesar datos
        print("📊 Aplicando transformaciones...")
        df_processed = processor.transformer.transform(df_raw)
        
        print(f"✅ Procesamiento completado")
        print(f"📐 Datos originales: {df_raw.shape}")
        print(f"📐 Datos procesados: {df_processed.shape}")
        
        # Generar estadísticas
        stats = processor.get_summary_statistics(df_processed)
        print(f"\n📈 ESTADÍSTICAS DEL PROCESAMIENTO:")
        print(f"   📊 Total de registros: {stats['total_records']:,}")
        print(f"   📅 Rango de fechas: {stats['date_range']['min']} a {stats['date_range']['max']}")
        print(f"   🌍 Regiones: {stats['regions_count']}")
        print(f"   📆 Años: {stats['years_count']}")
        print(f"   ❓ Valores faltantes: {stats['missing_values']}")
        
    except NameError:
        print("⚠️ Usando procesamiento básico (módulo personalizado no disponible)")
        
        # Procesamiento básico manual
        df_processed = df_raw.copy()
        
        # Limpiar nombres de columnas
        df_processed.columns = df_processed.columns.str.strip().str.lower()
        
        # Convertir valores numéricos
        if 'value' in df_processed.columns:
            df_processed['value'] = pd.to_numeric(df_processed['value'], errors='coerce')
        
        print(f"✅ Procesamiento básico completado")
        print(f"📐 Forma final: {df_processed.shape}")
    
    print("\n" + "="*50)

## 4. 🔍 Análisis Exploratorio de Datos (EDA)

Exploración detallada de las variables clave de la fuerza laboral chilena, incluyendo estadísticas descriptivas, detección de patrones y análisis de outliers.

In [ ]:
# Análisis descriptivo de variables principales
if 'df_processed' in locals():
    print("📊 ANÁLISIS DESCRIPTIVO")
    print("=" * 50)
    
    # Análisis de la variable principal (Value/Fuerza de Trabajo)
    value_col = 'value' if 'value' in df_processed.columns else df_processed.select_dtypes(include=[np.number]).columns[0]
    
    print(f"📈 Variable analizada: {value_col}")
    print(f"📊 Estadísticas descriptivas:")
    
    # Estadísticas básicas
    stats_basic = df_processed[value_col].describe()
    print(f"   Media: {stats_basic['mean']:,.0f}")
    print(f"   Mediana: {stats_basic['50%']:,.0f}")
    print(f"   Desviación estándar: {stats_basic['std']:,.0f}")
    print(f"   Mínimo: {stats_basic['min']:,.0f}")
    print(f"   Máximo: {stats_basic['max']:,.0f}")
    
    # Coeficiente de variación
    cv = (stats_basic['std'] / stats_basic['mean']) * 100
    print(f"   Coeficiente de variación: {cv:.2f}%")
    
    # Análisis por dimensiones
    print(f"\n🌍 ANÁLISIS POR DIMENSIONES:")
    
    # Por región (si existe)
    region_cols = [col for col in df_processed.columns if 'region' in col.lower()]
    if region_cols:
        region_col = region_cols[0]
        print(f"\n📍 Por {region_col}:")
        region_stats = df_processed.groupby(region_col)[value_col].agg(['count', 'mean', 'std']).round(0)
        display(region_stats.head(10))
    
    # Por género (si existe)
    gender_cols = [col for col in df_processed.columns if any(x in col.lower() for x in ['sexo', 'gender'])]
    if gender_cols:
        gender_col = gender_cols[0]
        print(f"\n👥 Por {gender_col}:")
        gender_stats = df_processed.groupby(gender_col)[value_col].agg(['count', 'mean', 'std']).round(0)
        display(gender_stats)
    
    # Por tiempo (si existe fecha)
    date_cols = [col for col in df_processed.columns if any(x in col.lower() for x in ['date', 'year', 'fecha', 'año'])]
    if date_cols:
        time_col = date_cols[0]
        print(f"\n📅 Tendencia temporal ({time_col}):")
        if 'year' in time_col.lower():
            time_stats = df_processed.groupby(time_col)[value_col].agg(['count', 'mean']).round(0)
            display(time_stats.tail(10))  # Últimos 10 años
    
    print("\n" + "="*50)
else:
    print("⚠️ Datos procesados no disponibles para análisis")

In [ ]:
# Detección de outliers y análisis de calidad
if 'df_processed' in locals():
    print("🎯 DETECCIÓN DE OUTLIERS Y CALIDAD DE DATOS")
    print("=" * 50)
    
    value_col = 'value' if 'value' in df_processed.columns else df_processed.select_dtypes(include=[np.number]).columns[0]
    
    # Método IQR para detección de outliers
    Q1 = df_processed[value_col].quantile(0.25)
    Q3 = df_processed[value_col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers_iqr = df_processed[(df_processed[value_col] < lower_bound) | 
                               (df_processed[value_col] > upper_bound)]
    
    print(f"📊 Detección de outliers (Método IQR):")
    print(f"   Rango normal: {lower_bound:,.0f} - {upper_bound:,.0f}")
    print(f"   Outliers encontrados: {len(outliers_iqr)} ({len(outliers_iqr)/len(df_processed)*100:.2f}%)")
    
    # Método Z-Score para outliers extremos
    z_scores = np.abs(stats.zscore(df_processed[value_col]))
    outliers_zscore = df_processed[z_scores > 3]
    
    print(f"\n📊 Outliers extremos (Z-score > 3):")
    print(f"   Outliers extremos: {len(outliers_zscore)} ({len(outliers_zscore)/len(df_processed)*100:.2f}%)")
    
    if len(outliers_zscore) > 0:
        print(f"   Valores extremos:")
        extreme_values = outliers_zscore[value_col].sort_values(ascending=False)
        for i, (idx, val) in enumerate(extreme_values.head(5).items()):
            print(f"      {i+1}. {val:,.0f}")
    
    # Análisis de distribución
    print(f"\n📈 Análisis de distribución:")
    
    # Test de normalidad (Shapiro-Wilk para muestras pequeñas, Anderson-Darling para grandes)
    if len(df_processed) <= 5000:
        shapiro_stat, shapiro_p = stats.shapiro(df_processed[value_col].dropna().sample(min(5000, len(df_processed))))
        print(f"   Test de Shapiro-Wilk: estadístico={shapiro_stat:.4f}, p-valor={shapiro_p:.6f}")
        is_normal = shapiro_p > 0.05
    else:
        # Para muestras grandes, usar test de Anderson-Darling
        anderson_result = stats.anderson(df_processed[value_col].dropna())
        print(f"   Test de Anderson-Darling: estadístico={anderson_result.statistic:.4f}")
        is_normal = anderson_result.statistic < anderson_result.critical_values[2]  # 5% level
    
    print(f"   ¿Distribución normal?: {'Sí' if is_normal else 'No'}")
    
    # Skewness y Kurtosis
    skewness = stats.skew(df_processed[value_col].dropna())
    kurtosis = stats.kurtosis(df_processed[value_col].dropna())
    
    print(f"   Asimetría (skewness): {skewness:.3f}")
    print(f"   Curtosis (kurtosis): {kurtosis:.3f}")
    
    # Interpretación
    if abs(skewness) < 0.5:
        skew_interp = "aproximadamente simétrica"
    elif abs(skewness) < 1:
        skew_interp = "moderadamente asimétrica"
    else:
        skew_interp = "altamente asimétrica"
    
    print(f"   Interpretación: Distribución {skew_interp}")
    
    print("\n" + "="*50)
else:
    print("⚠️ Datos procesados no disponibles para análisis de outliers")

## 5. 📊 Visualización Profesional de la Fuerza de Trabajo

Creación de gráficos ejecutivos y dashboards interactivos para comunicar los hallazgos de manera clara y profesional.

In [ ]:
# Visualizaciones básicas con matplotlib y seaborn
if 'df_processed' in locals():
    print("📈 CREANDO VISUALIZACIONES PROFESIONALES")
    print("=" * 50)
    
    value_col = 'value' if 'value' in df_processed.columns else df_processed.select_dtypes(include=[np.number]).columns[0]
    
    # Configuración de estilo
    plt.style.use('seaborn-v0_8')
    fig_size = (15, 10)
    
    # 1. Distribución de la variable principal
    plt.figure(figsize=fig_size)
    
    # Subplot 1: Histograma
    plt.subplot(2, 2, 1)
    plt.hist(df_processed[value_col] / 1000, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    plt.title('Distribución de la Fuerza de Trabajo', fontsize=14, fontweight='bold')
    plt.xlabel('Miles de personas')
    plt.ylabel('Frecuencia')
    plt.grid(True, alpha=0.3)
    
    # Subplot 2: Box plot
    plt.subplot(2, 2, 2)
    plt.boxplot(df_processed[value_col] / 1000)
    plt.title('Diagrama de Caja', fontsize=14, fontweight='bold')
    plt.ylabel('Miles de personas')
    plt.grid(True, alpha=0.3)
    
    # Subplot 3: Q-Q Plot
    plt.subplot(2, 2, 3)
    stats.probplot(df_processed[value_col].dropna(), dist="norm", plot=plt)
    plt.title('Q-Q Plot (Test de Normalidad)', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    
    # Subplot 4: Análisis temporal (si existe)
    plt.subplot(2, 2, 4)
    
    # Buscar columna de tiempo
    time_cols = [col for col in df_processed.columns if any(x in col.lower() for x in ['date', 'year', 'fecha', 'año'])]\n    
    if time_cols:
        time_col = time_cols[0]
        
        # Agrupar por tiempo y calcular promedio
        if 'date' in time_col.lower():
            time_data = df_processed.groupby(time_col)[value_col].mean() / 1000
        else:
            time_data = df_processed.groupby(time_col)[value_col].mean() / 1000
        
        plt.plot(time_data.index, time_data.values, marker='o', linewidth=2, color='coral')
        plt.title('Evolución Temporal', fontsize=14, fontweight='bold')
        plt.xlabel('Tiempo')
        plt.ylabel('Miles de personas (promedio)')
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3)
    else:
        plt.text(0.5, 0.5, 'No hay datos temporales\\ndisponibles', 
                ha='center', va='center', transform=plt.gca().transAxes,
                fontsize=12, bbox=dict(boxstyle='round', facecolor='lightgray'))
        plt.title('Evolución Temporal', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Visualizaciones básicas completadas")
else:
    print("⚠️ Datos no disponibles para visualización")

In [ ]:
# Visualizaciones por dimensiones (región, género, tiempo)
if 'df_processed' in locals():
    print("🌍 ANÁLISIS POR DIMENSIONES")
    print("=" * 50)
    
    value_col = 'value' if 'value' in df_processed.columns else df_processed.select_dtypes(include=[np.number]).columns[0]
    
    # Análisis por región
    region_cols = [col for col in df_processed.columns if 'region' in col.lower()]
    if region_cols:
        region_col = region_cols[0]
        
        # Top 10 regiones por fuerza de trabajo promedio
        plt.figure(figsize=(14, 8))
        
        region_avg = df_processed.groupby(region_col)[value_col].mean().sort_values(ascending=True) / 1000
        top_regions = region_avg.tail(10)
        
        plt.subplot(1, 2, 1)
        bars = plt.barh(range(len(top_regions)), top_regions.values, color='lightcoral')
        plt.yticks(range(len(top_regions)), top_regions.index)
        plt.title('Top 10 Regiones\\n(Fuerza de Trabajo Promedio)', fontsize=14, fontweight='bold')
        plt.xlabel('Miles de personas')
        plt.grid(True, alpha=0.3, axis='x')
        
        # Añadir valores en las barras
        for i, bar in enumerate(bars):
            width = bar.get_width()
            plt.text(width + width*0.01, bar.get_y() + bar.get_height()/2, 
                    f'{width:.0f}', ha='left', va='center', fontsize=10)
        
        # Distribución de varianza regional
        plt.subplot(1, 2, 2)
        region_std = df_processed.groupby(region_col)[value_col].std() / 1000
        plt.scatter(region_avg.values, region_std.values, alpha=0.6, s=60, color='darkblue')
        plt.xlabel('Promedio (miles de personas)')
        plt.ylabel('Desviación estándar (miles)')
        plt.title('Variabilidad Regional\\n(Promedio vs Varianza)', fontsize=14, fontweight='bold')
        plt.grid(True, alpha=0.3)
        
        # Añadir línea de tendencia
        z = np.polyfit(region_avg.values, region_std.values, 1)
        p = np.poly1d(z)
        plt.plot(region_avg.values, p(region_avg.values), \"r--\", alpha=0.8)
        
        plt.tight_layout()
        plt.show()
    
    # Análisis por género
    gender_cols = [col for col in df_processed.columns if any(x in col.lower() for x in ['sexo', 'gender'])]
    if gender_cols:
        gender_col = gender_cols[0]
        
        plt.figure(figsize=(12, 6))
        
        # Comparación por género
        plt.subplot(1, 2, 1)
        gender_data = df_processed.groupby(gender_col)[value_col].mean() / 1000
        colors = ['lightblue', 'lightpink', 'lightgreen'][:len(gender_data)]
        
        bars = plt.bar(gender_data.index, gender_data.values, color=colors, alpha=0.8, edgecolor='black')
        plt.title('Fuerza de Trabajo por Género\\n(Promedio)', fontsize=14, fontweight='bold')
        plt.ylabel('Miles de personas')
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3, axis='y')
        
        # Añadir valores en las barras
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                    f'{height:.0f}', ha='center', va='bottom', fontsize=11, fontweight='bold')
        
        # Box plot por género
        plt.subplot(1, 2, 2)
        gender_groups = [df_processed[df_processed[gender_col] == gender][value_col] / 1000 
                        for gender in df_processed[gender_col].unique()]
        
        plt.boxplot(gender_groups, labels=df_processed[gender_col].unique())
        plt.title('Distribución por Género', fontsize=14, fontweight='bold')
        plt.ylabel('Miles de personas')
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    print("✅ Análisis dimensional completado")
else:
    print("⚠️ Datos no disponibles para análisis dimensional")

In [ ]:
# Dashboard interactivo con Plotly
if 'df_processed' in locals():
    print("🚀 DASHBOARD INTERACTIVO")
    print("=" * 50)
    
    value_col = 'value' if 'value' in df_processed.columns else df_processed.select_dtypes(include=[np.number]).columns[0]
    
    try:
        # Crear dashboard usando módulo personalizado
        chart_maker = LabourForceCharts()
        interactive_fig = chart_maker.create_interactive_dashboard(
            df_processed, 
            title="Dashboard Interactivo - Fuerza de Trabajo Chile"
        )
        interactive_fig.show()
        print("✅ Dashboard interactivo creado con módulo personalizado")
        
    except NameError:
        print("⚠️ Creando dashboard básico (módulo personalizado no disponible)")
        
        # Dashboard básico con Plotly
        from plotly.subplots import make_subplots
        
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Distribución General', 'Evolución Temporal', 
                          'Por Región (Top 10)', 'Por Género'),
            specs=[[{"secondary_y": False}, {"secondary_y": False}],
                   [{"secondary_y": False}, {"secondary_y": False}]]
        )
        
        # 1. Histograma de distribución
        fig.add_trace(
            go.Histogram(x=df_processed[value_col] / 1000, 
                        name='Distribución', 
                        nbinsx=30,
                        marker_color='lightblue'),
            row=1, col=1
        )
        
        # 2. Evolución temporal (si existe)
        time_cols = [col for col in df_processed.columns if any(x in col.lower() for x in ['date', 'year', 'fecha', 'año'])]
        if time_cols:
            time_col = time_cols[0]
            time_data = df_processed.groupby(time_col)[value_col].mean() / 1000
            
            fig.add_trace(
                go.Scatter(x=time_data.index, y=time_data.values,
                          mode='lines+markers',
                          name='Evolución',
                          line=dict(color='coral', width=2)),
                row=1, col=2
            )
        
        # 3. Top regiones (si existe)
        region_cols = [col for col in df_processed.columns if 'region' in col.lower()]
        if region_cols:
            region_col = region_cols[0]
            region_data = df_processed.groupby(region_col)[value_col].mean().sort_values(ascending=True).tail(10) / 1000
            
            fig.add_trace(
                go.Bar(y=region_data.index, x=region_data.values,
                      orientation='h',
                      name='Regiones',
                      marker_color='lightgreen'),
                row=2, col=1
            )
        
        # 4. Por género (si existe)
        gender_cols = [col for col in df_processed.columns if any(x in col.lower() for x in ['sexo', 'gender'])]
        if gender_cols:
            gender_col = gender_cols[0]
            gender_data = df_processed.groupby(gender_col)[value_col].mean() / 1000
            
            fig.add_trace(
                go.Bar(x=gender_data.index, y=gender_data.values,
                      name='Género',
                      marker_color='lightcoral'),
                row=2, col=2
            )
        
        # Configurar layout
        fig.update_layout(
            title_text="Dashboard Fuerza de Trabajo Chile",
            height=800,
            showlegend=False,
            template='plotly_white'
        )
        
        # Actualizar etiquetas de ejes
        fig.update_xaxes(title_text="Miles de personas", row=1, col=1)
        fig.update_yaxes(title_text="Frecuencia", row=1, col=1)
        
        if time_cols:
            fig.update_xaxes(title_text="Tiempo", row=1, col=2)
            fig.update_yaxes(title_text="Miles de personas", row=1, col=2)
        
        if region_cols:
            fig.update_xaxes(title_text="Miles de personas", row=2, col=1)
            fig.update_yaxes(title_text="Región", row=2, col=1)
        
        if gender_cols:
            fig.update_xaxes(title_text="Género", row=2, col=2)
            fig.update_yaxes(title_text="Miles de personas", row=2, col=2)
        
        fig.show()
        print("✅ Dashboard básico completado")
    
    print("\n💡 El dashboard es interactivo: puedes hacer zoom, filtrar y explorar los datos")
    print("=" * 50)
else:
    print("⚠️ Datos no disponibles para dashboard")

## 6. 🧠 Generación de Insights y Modelos Predictivos

Identificación de patrones clave, generación de insights económicos y construcción de modelos predictivos cuando los datos lo permiten.

In [ ]:
# Generación de insights principales
if 'df_processed' in locals():
    print("💡 INSIGHTS ECONÓMICOS PRINCIPALES")
    print("=" * 60)
    
    value_col = 'value' if 'value' in df_processed.columns else df_processed.select_dtypes(include=[np.number]).columns[0]
    insights = []
    
    # 1. Insight general sobre magnitud
    total_workforce = df_processed[value_col].sum()
    avg_workforce = df_processed[value_col].mean()
    
    insights.append(f"📊 La fuerza de trabajo total analizada alcanza {total_workforce/1000000:.1f} millones de personas")
    insights.append(f"📈 El promedio por observación es de {avg_workforce/1000:.0f} mil personas")
    
    # 2. Insights temporales
    time_cols = [col for col in df_processed.columns if any(x in col.lower() for x in ['date', 'year', 'fecha', 'año'])]
    if time_cols:
        time_col = time_cols[0]
        
        # Calcular tendencia temporal
        time_data = df_processed.groupby(time_col)[value_col].sum()
        
        if len(time_data) > 2:
            # Calcular tasa de crecimiento
            first_period = time_data.iloc[0]
            last_period = time_data.iloc[-1]
            periods = len(time_data) - 1
            
            growth_rate = ((last_period / first_period) ** (1/periods) - 1) * 100
            total_growth = ((last_period / first_period) - 1) * 100
            
            insights.append(f"📅 Tendencia temporal: crecimiento promedio anual del {growth_rate:.2f}%")
            insights.append(f"📈 Crecimiento total en el período: {total_growth:.1f}%")
            
            # Identificar período de mayor/menor actividad
            max_period = time_data.idxmax()
            min_period = time_data.idxmin()
            
            insights.append(f"🔝 Período de mayor actividad: {max_period} ({time_data[max_period]/1000000:.2f}M personas)")
            insights.append(f"🔻 Período de menor actividad: {min_period} ({time_data[min_period]/1000000:.2f}M personas)")
    
    # 3. Insights regionales
    region_cols = [col for col in df_processed.columns if 'region' in col.lower()]
    if region_cols:
        region_col = region_cols[0]
        
        region_totals = df_processed.groupby(region_col)[value_col].sum().sort_values(ascending=False)
        
        # Región líder
        top_region = region_totals.index[0]
        top_value = region_totals.iloc[0]
        
        # Concentración regional
        top_3_share = region_totals.head(3).sum() / region_totals.sum() * 100
        
        insights.append(f"🌍 Región líder: {top_region} ({top_value/1000000:.1f}M personas)")
        insights.append(f"🎯 Las top 3 regiones concentran el {top_3_share:.1f}% de la fuerza laboral")
        
        # Disparidad regional
        region_cv = (region_totals.std() / region_totals.mean()) * 100
        insights.append(f"📊 Disparidad regional (CV): {region_cv:.1f}% ({'Alta' if region_cv > 50 else 'Moderada' if region_cv > 30 else 'Baja'})")
    
    # 4. Insights de género
    gender_cols = [col for col in df_processed.columns if any(x in col.lower() for x in ['sexo', 'gender'])]
    if gender_cols:
        gender_col = gender_cols[0]
        
        gender_totals = df_processed.groupby(gender_col)[value_col].sum()
        
        if 'M' in gender_totals.index and 'F' in gender_totals.index:
            male_total = gender_totals.get('M', 0)
            female_total = gender_totals.get('F', 0)
            
            if male_total > 0 and female_total > 0:
                gender_ratio = male_total / female_total
                female_participation = female_total / (male_total + female_total) * 100
                
                insights.append(f"👥 Participación femenina: {female_participation:.1f}% del total")
                insights.append(f"⚖️ Ratio hombres/mujeres: {gender_ratio:.2f}")
                
                if gender_ratio > 1.5:
                    insights.append("📊 Existe una brecha significativa de género en participación laboral")
                elif gender_ratio < 1.1:
                    insights.append("📊 La participación laboral está relativamente equilibrada por género")
    
    # 5. Insights de variabilidad
    cv = (df_processed[value_col].std() / df_processed[value_col].mean()) * 100
    insights.append(f"📈 Variabilidad general (CV): {cv:.1f}% ({'Alta' if cv > 30 else 'Moderada' if cv > 15 else 'Baja'})")
    
    # Mostrar insights
    print("🔍 PRINCIPALES HALLAZGOS:")
    for i, insight in enumerate(insights, 1):
        print(f"{i:2d}. {insight}")
    
    print("\\n" + "="*60)
    
    # Generar recomendaciones
    print("\\n🎯 RECOMENDACIONES ESTRATÉGICAS:")
    recommendations = []
    
    if 'growth_rate' in locals() and growth_rate > 2:
        recommendations.append("📈 El crecimiento sostenido sugiere políticas de empleo efectivas")
    elif 'growth_rate' in locals() and growth_rate < 0:
        recommendations.append("⚠️ El decrecimiento requiere políticas de estímulo al empleo")
    
    if 'region_cv' in locals() and region_cv > 50:
        recommendations.append("🌍 Alta disparidad regional requiere políticas de desarrollo territorial")
    
    if 'female_participation' in locals() and female_participation < 40:
        recommendations.append("👩 Baja participación femenina sugiere necesidad de políticas de inclusión")
    
    if cv > 30:
        recommendations.append("📊 Alta variabilidad indica mercado laboral heterogéneo")
    
    if not recommendations:
        recommendations.append("📊 Los indicadores muestran un mercado laboral relativamente estable")
    
    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. {rec}")
    
    print("\\n" + "="*60)
else:
    print("⚠️ Datos no disponibles para generación de insights")

In [ ]:
# Modelo predictivo básico (si hay datos temporales)
if 'df_processed' in locals():
    print("🤖 MODELO PREDICTIVO BÁSICO")
    print("=" * 50)
    
    time_cols = [col for col in df_processed.columns if any(x in col.lower() for x in ['date', 'year', 'fecha', 'año'])]
    value_col = 'value' if 'value' in df_processed.columns else df_processed.select_dtypes(include=[np.number]).columns[0]
    
    if time_cols:
        time_col = time_cols[0]
        
        # Preparar datos para modelado
        # Filtrar datos nacionales para el modelo
        national_filter = df_processed['region_code'] == '_T' if 'region_code' in df_processed.columns else True
        gender_filter = df_processed['gender_code'] == '_T' if 'gender_code' in df_processed.columns else True
        
        if isinstance(national_filter, bool):
            model_data = df_processed[gender_filter] if not isinstance(gender_filter, bool) else df_processed
        else:
            model_data = df_processed[national_filter & gender_filter] if not isinstance(gender_filter, bool) else df_processed[national_filter]
        
        if len(model_data) > 5:  # Necesitamos suficientes datos
            # Crear series temporal
            ts_data = model_data.groupby(time_col)[value_col].mean().sort_index()
            
            print(f"📊 Datos para modelado: {len(ts_data)} observaciones temporales")
            
            # Modelo de tendencia lineal simple
            from sklearn.linear_model import LinearRegression
            from sklearn.metrics import mean_absolute_error, r2_score
            
            # Preparar variables
            X = np.arange(len(ts_data)).reshape(-1, 1)  # Tiempo como variable numérica
            y = ts_data.values
            
            # Dividir en entrenamiento y prueba (80-20)
            split_idx = int(len(X) * 0.8)
            X_train, X_test = X[:split_idx], X[split_idx:]
            y_train, y_test = y[:split_idx], y[split_idx:]
            
            # Entrenar modelo
            model = LinearRegression()
            model.fit(X_train, y_train)
            
            # Predicciones
            y_pred_train = model.predict(X_train)
            y_pred_test = model.predict(X_test) if len(X_test) > 0 else []
            
            # Métricas
            train_r2 = r2_score(y_train, y_pred_train)
            train_mae = mean_absolute_error(y_train, y_pred_train)
            
            print(f"✅ Modelo entrenado exitosamente")
            print(f"📈 R² (entrenamiento): {train_r2:.3f}")
            print(f"📊 MAE (entrenamiento): {train_mae/1000:.0f} miles de personas")
            
            if len(y_test) > 0:
                test_r2 = r2_score(y_test, y_pred_test)
                test_mae = mean_absolute_error(y_test, y_pred_test)
                print(f"🎯 R² (prueba): {test_r2:.3f}")
                print(f"📊 MAE (prueba): {test_mae/1000:.0f} miles de personas")
            
            # Proyección futura (próximos 4 períodos)
            future_periods = 4
            X_future = np.arange(len(ts_data), len(ts_data) + future_periods).reshape(-1, 1)
            y_future = model.predict(X_future)
            
            print(f"\\n🔮 PROYECCIONES FUTURAS ({future_periods} períodos):")
            for i, pred in enumerate(y_future, 1):
                print(f"   Período +{i}: {pred/1000:.0f} miles de personas")
            
            # Visualización del modelo
            plt.figure(figsize=(12, 6))
            
            # Datos históricos
            plt.plot(range(len(ts_data)), ts_data.values/1000, 'o-', label='Datos reales', color='blue', alpha=0.7)
            
            # Predicciones del modelo
            plt.plot(range(len(y_train)), y_pred_train/1000, '--', label='Predicción (entrenamiento)', color='red', alpha=0.8)
            
            if len(y_test) > 0:
                plt.plot(range(len(y_train), len(ts_data)), y_pred_test/1000, '--', label='Predicción (prueba)', color='orange', alpha=0.8)
            
            # Proyecciones futuras
            future_x = range(len(ts_data), len(ts_data) + future_periods)
            plt.plot(future_x, y_future/1000, 's--', label='Proyección futura', color='green', alpha=0.8)
            
            # Configuración del gráfico
            plt.title('Modelo Predictivo de Fuerza de Trabajo', fontsize=14, fontweight='bold')
            plt.xlabel('Período')
            plt.ylabel('Miles de personas')
            plt.legend()
            plt.grid(True, alpha=0.3)
            
            # Añadir línea divisoria entre datos y proyección
            plt.axvline(x=len(ts_data)-0.5, color='gray', linestyle=':', alpha=0.7, label='Inicio proyección')
            
            plt.tight_layout()
            plt.show()
            
            # Interpretación del modelo
            slope = model.coef_[0]
            trend_interpretation = "creciente" if slope > 0 else "decreciente" if slope < 0 else "estable"
            
            print(f"\\n📊 INTERPRETACIÓN DEL MODELO:")
            print(f"   Tendencia: {trend_interpretation}")
            print(f"   Cambio promedio por período: {slope/1000:+.0f} miles de personas")
            
            if train_r2 > 0.8:
                print(f"   Calidad del modelo: Excelente (R² = {train_r2:.3f})")
            elif train_r2 > 0.6:
                print(f"   Calidad del modelo: Buena (R² = {train_r2:.3f})")
            elif train_r2 > 0.4:
                print(f"   Calidad del modelo: Moderada (R² = {train_r2:.3f})")
            else:
                print(f"   Calidad del modelo: Baja (R² = {train_r2:.3f}) - datos muy variables")
            
        else:
            print("⚠️ Insuficientes datos temporales para modelado")
    else:
        print("⚠️ No se encontraron columnas temporales para modelado predictivo")
        print("💡 Sugerencia: Incluir variables temporales para habilitar predicciones")
    
    print("\\n" + "="*50)
else:
    print("⚠️ Datos no disponibles para modelado")

## 7. 🏗️ Buenas Prácticas y Estructura Modular del Proyecto

Demostración de la organización profesional del código, aplicación de principios SOLID y Clean Code, y documentación de la arquitectura del proyecto.

In [ ]:
# Documentación de la estructura del proyecto
print("🏗️ ESTRUCTURA MODULAR DEL PROYECTO")
print("=" * 60)

print("""
📁 ine-chile-labour-force-analysis/
├── 📊 data/
│   ├── raw/           # Datos crudos del INE (CSV originales)
│   ├── processed/     # Datos limpios y transformados
│   └── outputs/       # Resultados finales y exportaciones
├── 📝 src/           # Código fuente modular
│   ├── etl/          # Extract, Transform, Load
│   │   ├── __init__.py
│   │   ├── base.py           # Clases base abstractas
│   │   └── data_processor.py # Procesador de datos del INE
│   ├── models/       # Modelos estadísticos y ML
│   │   └── __init__.py
│   ├── utils/        # Utilidades y helpers
│   │   ├── __init__.py
│   │   ├── logger_config.py  # Configuración de logging
│   │   └── validators.py     # Validadores de datos
│   └── visualization/ # Gráficos y dashboards
│       ├── __init__.py
│       └── charts.py         # Clases para visualización
├── 📔 notebooks/     # Jupyter notebooks de análisis
│   └── 01_eda_labour_force.ipynb
├── 🔧 scripts/       # Scripts de automatización
├── 🧪 tests/         # Tests unitarios e integración
├── 📚 docs/          # Documentación del proyecto
├── 📋 config.py      # Configuración centralizada
├── 📄 requirements.txt # Dependencias
├── ⚙️ setup.py       # Configuración del paquete
└── 📖 README.md      # Documentación principal
""")

print("🎯 PRINCIPIOS APLICADOS:")
print("-" * 40)

principles = [
    ("Clean Code", [
        "Nombres descriptivos y funciones pequeñas",
        "Principio DRY (Don't Repeat Yourself)", 
        "Comentarios claros y documentación exhaustiva",
        "Código auto-documentado y legible"
    ]),
    
    ("SOLID Principles", [
        "S - Single Responsibility: Cada clase tiene una responsabilidad",
        "O - Open/Closed: Abierto para extensión, cerrado para modificación",
        "L - Liskov Substitution: Clases derivadas son sustituibles",
        "I - Interface Segregation: Interfaces específicas y cohesivas",
        "D - Dependency Inversion: Depender de abstracciones, no concreciones"
    ]),
    
    ("Data Science Best Practices", [
        "Validación exhaustiva de datos",
        "Logging y monitoreo de procesos",
        "Reproducibilidad de experimentos",
        "Versionado de datos y código",
        "Documentación de metodología"
    ])
]

for principle, practices in principles:
    print(f"\\n📚 {principle}:")
    for practice in practices:
        print(f"   ✅ {practice}")

print("\\n" + "="*60)

In [ ]:
# Ejemplo de uso de módulos profesionales y guardar resultados
print("💾 GUARDADO DE RESULTADOS Y EJEMPLO DE MODULARIEDAD")
print("=" * 60)

# Guardar datos procesados
if 'df_processed' in locals():
    
    # Crear directorio de salida si no existe
    output_path = processed_data_path / "labour_force_processed.csv"
    
    try:
        # Guardar datos procesados
        df_processed.to_csv(output_path, index=False, encoding='utf-8')
        print(f"✅ Datos procesados guardados en: {output_path}")
        print(f"📊 Registros guardados: {len(df_processed):,}")
        
        # Guardar resumen estadístico
        summary_path = output_data_path / "summary_statistics.txt"
        
        with open(summary_path, 'w', encoding='utf-8') as f:
            f.write("RESUMEN ESTADÍSTICO - FUERZA DE TRABAJO CHILE\\n")
            f.write("=" * 50 + "\\n\\n")
            
            f.write(f"Fecha de análisis: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\\n")
            f.write(f"Total de registros: {len(df_processed):,}\\n")
            f.write(f"Período analizado: {df_processed['date'].min()} a {df_processed['date'].max()}\\n" if 'date' in df_processed.columns else "")
            f.write(f"Regiones incluidas: {df_processed['region_code'].nunique()}\\n" if 'region_code' in df_processed.columns else "")
            
            # Estadísticas principales
            value_col = 'value' if 'value' in df_processed.columns else df_processed.select_dtypes(include=[np.number]).columns[0]
            stats = df_processed[value_col].describe()
            
            f.write("\\nESTADÍSTICAS PRINCIPALES:\\n")
            f.write(f"Promedio: {stats['mean']:,.0f}\\n")
            f.write(f"Mediana: {stats['50%']:,.0f}\\n")
            f.write(f"Mínimo: {stats['min']:,.0f}\\n")
            f.write(f"Máximo: {stats['max']:,.0f}\\n")
            f.write(f"Desviación estándar: {stats['std']:,.0f}\\n")
        
        print(f"✅ Resumen estadístico guardado en: {summary_path}")
        
    except Exception as e:
        print(f"❌ Error al guardar archivos: {str(e)}")

# Ejemplo de uso modular profesional
print("\\n🔧 EJEMPLO DE USO MODULAR:")
print("-" * 40)

code_example = '''
# Ejemplo de uso profesional de los módulos

# 1. Importar módulos del proyecto
from etl.data_processor import LabourForceProcessor
from visualization.charts import LabourForceCharts
from utils.validators import LabourForceValidator

# 2. Crear pipeline de procesamiento
processor = LabourForceProcessor()

# 3. Procesar datos con validación
raw_data = processor.extractor.extract("data/raw/labour_force.csv")
clean_data = processor.transformer.transform(raw_data)
processor.loader.load(clean_data, "data/processed/labour_force_clean.csv")

# 4. Crear visualizaciones profesionales
charts = LabourForceCharts()
fig = charts.plot_time_series(clean_data, save_path="outputs/time_series.png")
dashboard = charts.create_interactive_dashboard(clean_data)

# 5. Validar calidad de datos
validator = LabourForceValidator()
validation_results = validator.validate_labour_force_data(clean_data)
report = validator.get_validation_report()
'''

print(code_example)

print("🎯 VENTAJAS DE LA ARQUITECTURA MODULAR:")
print("-" * 40)

advantages = [
    "🔄 Reutilización: Los módulos pueden ser reutilizados en otros proyectos",
    "🧪 Testabilidad: Cada módulo puede ser testeado independientemente", 
    "🔧 Mantenibilidad: Cambios localizados no afectan todo el sistema",
    "📈 Escalabilidad: Fácil añadir nuevas funcionalidades",
    "👥 Colaboración: Múltiples desarrolladores pueden trabajar en paralelo",
    "📚 Documentación: Cada módulo está auto-documentado",
    "🎨 Flexibilidad: Fácil intercambiar implementaciones",
    "🚀 Productividad: Menos código duplicado, más eficiencia"
]

for advantage in advantages:
    print(f"   {advantage}")

print("\\n" + "="*60)

# Información del proyecto para stakeholders
print("\\n📋 INFORMACIÓN PARA STAKEHOLDERS:")
print("-" * 40)

stakeholder_info = [
    ("🎯 Objetivo del Proyecto", "Análisis avanzado de la Fuerza de Trabajo en Chile usando datos oficiales del INE"),
    ("📊 Metodología", "Data Science con Clean Code y principios SOLID"),
    ("🔍 Análisis Incluidos", "Exploratorio, temporal, regional, por género, predictivo"),
    ("📈 Visualizaciones", "Gráficos ejecutivos, dashboards interactivos, mapas"),
    ("🤖 Modelos", "Predictivos para proyecciones futuras"),
    ("📚 Documentación", "Completa y profesional para reproducibilidad"),
    ("🔧 Arquitectura", "Modular y escalable para mantenimiento"),
    ("📄 Entregables", "Código, datos procesados, visualizaciones, insights")
]

for title, description in stakeholder_info:
    print(f"{title}: {description}")

print(f"\\n🏆 RESULTADO: Repositorio profesional evidenciando capacidades senior en Data Science")
print("=" * 60)

## 🎯 Conclusiones y Próximos Pasos

### 📊 Resumen Ejecutivo

Este análisis exploratorio de la Fuerza de Trabajo en Chile demuestra la aplicación de metodologías avanzadas de Data Science siguiendo las mejores prácticas de la industria:

### ✅ Logros Principales

1. **Análisis Completo**: Exploración exhaustiva de datos del INE con validación de calidad
2. **Arquitectura Profesional**: Implementación de Clean Code y principios SOLID
3. **Visualizaciones Ejecutivas**: Gráficos profesionales y dashboards interactivos
4. **Insights Económicos**: Identificación de patrones y tendencias clave
5. **Modelos Predictivos**: Proyecciones basadas en datos históricos
6. **Documentación Completa**: Código reproducible y bien documentado

### 🚀 Próximos Pasos

1. **Expansión de Datos**: Integrar más fuentes del INE (ocupación, desocupación, etc.)
2. **Dashboard Web**: Crear aplicación web interactiva con Streamlit/Dash
3. **Machine Learning Avanzado**: Implementar modelos de forecasting más sofisticados
4. **API de Datos**: Desarrollar API REST para consultas automáticas
5. **Automatización**: Crear pipelines automáticos de actualización de datos
6. **Análisis Sectorial**: Incluir análisis por sectores económicos
7. **Geolocalización**: Añadir mapas interactivos regionales

### 🏆 Valor del Proyecto

Este repositorio evidencia capacidades **senior en Data Science** a través de:
- Código limpio y arquitectura escalable
- Metodología científica rigurosa
- Visualizaciones de calidad ejecutiva
- Documentación profesional completa
- Aplicación de mejores prácticas de la industria

---

**📧 Contacto**: Bruno San Martin | **🔗 GitHub**: @SanMaBruno | **📅 Fecha**: Julio 2025